<H2 align="center"> Baixar o arquivo csv, caso tenha baixado o arquivo via github:</H2>


In [ ]:
import subprocess
import sys

# Tenta importar gdown, se não conseguir, instala
try:
    import gdown
except ImportError:
    print("📥 gdown não encontrado. Instalando automaticamente...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown  # Tenta importar novamente após a instalação

# Baixar o arquivo do Google Drive

file_id = "1VbB_fvowkgNoWcLADi1mEfFjWQI6nBjp"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dados/SpSafe_2022.csv"

gdown.download(url, output, quiet=False)


<p align="center">
  Leitura do arquivo
</p>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')
pd.set_option('display.max_columns', None)


<H2 align="center"> Códigos criados para adicionar uma coluna com os atributos dia da semana. </H2>
 

<P aling = "center">  </P>
<p align="center">
  Executar somente quando um novo arquivo for baixado do Google Drive
</p>



In [41]:
#Conventendo a coluna DATA_OCORRENCIA para datetime para ser reconhecido como data pelo pandas
df['DATA_OCORRENCIA'] = pd.to_datetime(df['DATA_OCORRENCIA'])

In [ ]:
#Criação de coluna com o dia da semana
df.insert(df.columns.get_loc('DATA_OCORRENCIA') + 1, 'DIA_SEMANA', df['DATA_OCORRENCIA'].dt.day_name())

In [43]:
#Convertendo os dias da semana para português
dias_semana = {
    'Monday': 'SEGUNDA-FEIRA', 'Tuesday': 'TERÇA-FEIRA', 'Wednesday': 'QUARTA-FEIRA',
    'Thursday': 'QUINTA-FEIRA', 'Friday': 'SEXTA-FEIRA', 'Saturday': 'SÁBADO', 'Sunday': 'DOMINGO'
}
df['DIA_SEMANA'] = df['DATA_OCORRENCIA'].dt.day_name().map(dias_semana)


In [ ]:
df.head(20)

<h2 align="center">
  Código criados para localizar as cidades dos dados via latitude e longitude
</h2>


<p align="center">
  Executar somente para o arquivo csv baixado via Google Drive
</p>


<div style = "text-align: center;">
<strong>tempo de processamento longo por causa da API da biblioteca geopy</strong>
</div>


In [9]:
# Biblioteca usada para localizar o nome da cidade via latitude e longitude
from geopy.geocoders import Nominatim
geolocal = Nominatim(user_agent="tpDados")


In [ ]:
# Código criado para preencher a coluna com o nome da cidade conforme latitude e longitude
for indice, linha in df.iterrows():
    if linha["LATITUDE"].notna() & linha["LONGITUDE"].notna() & linha["CIDADE"].notna():
        continue
    if pd.isna(linha["LATITUDE"]) or pd.isna(linha["LONGITUDE"]):  
        df.at[indice, "CIDADE"] = np.nan  
    else:
        latitude = linha["LATITUDE"]
        longitude = linha["LONGITUDE"]
        local = geolocal.reverse(f"{latitude}, {longitude}")
        df.at[indice, "CIDADE"] = local.raw["address"].get("city", np.nan)  



    

<p align="center">
  Comando para salvar o DataFrame em um arquivo csv
</p>

In [55]:
df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')